In [51]:
# CURRENTLY WORKING IN THIS ONE
import os
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from PIL import Image
import numpy as np
from random import randint
from sklearn import preprocessing as p
import gc
import cv2
import math


In [52]:
torch.cuda.empty_cache()
gc.collect()

1425

In [53]:
min_max_scaler = p.MinMaxScaler()
INPUT_DATA_PATH = 'test_input_new.npy'
OUTPUT_DATA_PATH = 'test_output_new.npy'
x_input = np.load(INPUT_DATA_PATH)
y_input = np.load(OUTPUT_DATA_PATH)

x_min = x_input.min(axis=(1, 2), keepdims=True)
x_max = x_input.max(axis=(1, 2), keepdims=True)
x_input = (x_input - x_min)/(x_max-x_min)

y_min = y_input.min(axis=(1, 2), keepdims=True)
y_max = y_input.max(axis=(1, 2), keepdims=True)
y_input = (y_input - y_min)/(y_max-y_min)

num_points = np.load('test_nums.npy')
# print(x_input)

In [54]:
# x_input[x_input < 1] = 0


# print(x_input[0][0])

In [55]:
def threshold(tensor):
    for img in range(len(tensor)):
            for channel in range(1):
                    for h in range(100):
                            for w in range(100):
                                    if (tensor[img][channel][h][w] < 1):
                                            tensor[img][channel][h][w] = 0

In [56]:
SPLIT_IDX = 7500
x_train = torch.tensor(x_input[:SPLIT_IDX], dtype=torch.float).permute(0,3,1,2)
x_test = torch.tensor(x_input[SPLIT_IDX:], dtype=torch.float).permute(0,3,1,2)

y_train = torch.tensor(y_input[:SPLIT_IDX], dtype=torch.float).permute(0,3,1,2)
y_test = torch.tensor(y_input[SPLIT_IDX:], dtype=torch.float).permute(0,3,1,2)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
x_train = x_train.to(device)
x_test = x_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)

test_num_points = num_points[SPLIT_IDX:]

# print (test_num_points.shape)

# threshold(x_train)                       
# threshold(x_test)
# threshold(y_train)                       
# threshold(y_test)


train_dataset = TensorDataset(x_train,y_train) # create your datset
# train_dataloader = DataLoader(train_dataset, shuffle=True) # create your dataloader

train_dataloader = DataLoader(train_dataset, batch_size=100, shuffle=True) # create your dataloader

print (len(train_dataloader))

test_dataset = TensorDataset(x_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=100, shuffle=False)

#/home/bwilab/asha_ritu/line_model/conv_network.ipynb


75


In [57]:
# create a CNN
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        self.cnn_stack = nn.Sequential( 
            # first layer: learning one person. 
            # kernel size = diameter of a dot (5 pixels) + padding (2 pixels) = (7,7)
            # out_channels: 1 for now
            # try changing the out_channels
            # larger dataset and ReLU between each layer
            # larger dataset, more layers, larger kernels

            # find center pixel of each circle, count number of dots

            # blob detect: looking for continuous regions of non-white pixels
            # confidence regions. 
            # masking the image like in hw3
            # 
            nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(6,6), stride=1, bias=True, padding=(2,2)), # input layer
            nn.ReLU(),

            # nn.MaxPool2d(kernel_size=(2,2), stride=2),

            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(11,11), stride=1, bias=True, padding=(5,5)), # input layer
            nn.ReLU(),

            # nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(19,19), stride=1, bias=True, padding=(9,9)), # input layer
            # nn.ReLU(),
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(11,11), stride=1, bias=True, padding=(5,5)), # input layer
            nn.ReLU(),


            # layer 2: learning two dots.
            # kernel_size = diameter of two dots (10 pixels) + distance between them (2 pixels) + padding (2 pixels)
            # there are 8 different orientations two people can be in, resulting in 8 output channels.
            # two options: either increase kernel size and number of channels or

            # pool -> lower image resolution and learn a smaller pattern

            # nn.MaxPool2d()


            # size of output channel: [(input size + kernel size + padding top + padding bottom) / stride] + 1

            # nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(12, 12), stride=1, bias=True, padding=(8,8)),
            # nn.ReLU(),
            # nn.MaxPool2d(kernel_size=(2,2), stride=2),


            # nn.Conv2d(in_channels=8, out_channels=10, kernel_size=(15, 15), stride=1, bias=True, padding=(8,8)),
            # nn.ReLU(),
            # nn.MaxPool2d(kernel_size=(2,2), stride=2),

            # final layer: out_channels=1, 
            # kernel size = ? ask Dr. Hart what a reasonable kernel size for the output layer could be.
            nn.Conv2d(in_channels=10, out_channels=1, kernel_size=(6,6), padding=(3,3)), # 100 x 100
            nn.ReLU()
            # nn.Sigmoid() # values between 0 and 1
        )

    def forward(self, x):
        logits = self.cnn_stack(x)
        return logits

In [58]:
model = NeuralNetwork()
if torch.cuda.is_available():
    model.cuda()
    
learning_rate = 3e-4
weight_decay=1e-5
optimizer = torch.optim.Adam(params=model.parameters(), weight_decay=weight_decay, lr=learning_rate)
batch_size = 16
epochs = 30
# Initialize the loss function
loss_fn = nn.MSELoss()
# loss_fn = nn.CrossEntropyLoss() # can change this to another loss function
test_losses = []

In [59]:
def train_loop(dataloader, model, loss_fn, optimizer):
    avg_loss = 0
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        optimizer.zero_grad()
                  
        # print(X)
        pred = model(X)
        # print(f"pred:{pred}")
        # print(f"target:{y}")
    
        # print(f"pred: {pred}")
        # print(f"y shape:{y.shape}")
        loss = loss_fn(pred, y)
        avg_loss += loss
        loss.backward()
        optimizer.step()
        
        # if batch % 100 == 0:
        #     print(f"loss: {loss:>7f}")
    avg_loss = avg_loss / size
    print(f"average loss: {avg_loss}")

        # if batch % 50 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [60]:
size = len(train_dataloader.dataset)
print(size)

7500


In [61]:
def mask(model_input, model_output):
    WHITE = 1
    BLACK = 0

    model_output = 1-model_output
    model_input = 1-model_input
    model_input[model_input != WHITE] = BLACK

    masked_arr = np.logical_and(model_input, model_output, model_input == WHITE)
    return masked_arr

In [62]:
def test():
    avg_accuracy = 0
    size = len(test_dataloader.dataset)
    i = 0
    with torch.no_grad():
        for data, target in test_dataloader:
            output = model(data)
            # print(output*255)
            for out_i in range(100):
                
                masked_arr = mask(np.array(data.cpu().data[out_i][0]), np.array(output.cpu().data[out_i][0]))
                num_dots = np.count_nonzero(masked_arr != 0.0)
                num_expected = test_num_points[i]
                
                curr_diff = (num_dots - num_expected)**2
                if (out_i % 50 == 0):
                    print(f"diff: {curr_diff}")
                # num_expected = np.count_nonzero(np.array(target.cpu().data[out_i][0]) == 0.0)

                # print(np.array(target.cpu().data[out_i][0]).shape)
                # if (i == t):
                #     print(f"num counted: {num_dots}, num expected:{num_expected}")

                #     im = Image.fromarray(np.array(data.cpu().data[out_i][0])*255)
                #     im.show()

                # print(f"accuracy of this trial: {(num_dots - num_expected)/num_expected * 100}")
                avg_accuracy += curr_diff
                i += 1
    avg_accuracy = math.sqrt(avg_accuracy / size)
    print(f"RMSE accuracy: {avg_accuracy}")

# test(500)

In [63]:
print(test_num_points[500])

4.0


In [64]:
# test()
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    # test(500)
    # test_loop(test_dataloader, model, loss_fn)
    
print("Done!")


Epoch 1
-------------------------------
average loss: 0.0008085418376140296
Epoch 2
-------------------------------
average loss: 9.089544619200751e-05
Epoch 3
-------------------------------
average loss: 3.119631946901791e-05
Epoch 4
-------------------------------
average loss: 2.2650814571534283e-05
Epoch 5
-------------------------------
average loss: 1.9488190446281806e-05
Epoch 6
-------------------------------
average loss: 1.8025946701527573e-05
Epoch 7
-------------------------------
average loss: 1.6625406715320423e-05
Epoch 8
-------------------------------
average loss: 1.5859830455156043e-05
Epoch 9
-------------------------------
average loss: 1.4666788956674282e-05
Epoch 10
-------------------------------
average loss: 1.3962377124698833e-05
Epoch 11
-------------------------------
average loss: 1.3503035006579012e-05
Epoch 12
-------------------------------
average loss: 1.3190375284466427e-05
Epoch 13
-------------------------------
average loss: 1.2384598448988982e-0

In [65]:
torch.save(model, 'model.pt')

In [66]:
# Model class must be defined somewhere
model = torch.load('model.pt')

In [67]:
i = 60

with torch.no_grad():
    for data, target in test_dataloader:
        output = model(data)    
        # INPUT
        input_arr = np.array(data.cpu().data[i][0])*255
        input = Image.fromarray(input_arr)
        input.show()

        # OUTPUT
        output_arr = np.array(output.cpu().data[i][0])*255
        output = Image.fromarray(output_arr)
        output.show()

        masked_arr = mask(output_arr/255, input_arr/255)
        masked_arr = masked_arr
        masked = Image.fromarray(masked_arr)
        masked.show()

        num_dots = np.count_nonzero(masked_arr != 0.0)
        num_expected = test_num_points[i]
        print(f"num counted: {num_dots}, num expected:{num_expected}")

        break

num counted: 0, num expected:8.0


In [68]:
# test()
# test()
# for t in range(epochs):
#     print(f"Epoch {t+1}\n-------------------------------")
#     # train_loop(train_dataloader, model, loss_fn, optimizer)
#     test()
#     # test_loop(test_dataloader, model, loss_fn)
    
# print("Done!")



(eog:8807): Gtk-WARNING **: 13:31:33.708: Could not load a pixbuf from icon theme.
This may indicate that pixbuf loaders or the mime database could not be found.
